In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [13]:
import os
import shutil
import numpy as np

# Define the base directory where your data is stored
base_dir = '/content/drive/MyDrive/Cervical cancer image dataset'

# Create directories for training, validation, and testing
train_dir = '/content/data/train'
val_dir = '/content/data/validation'
test_dir = '/content/data/test'

# Create subdirectories for each class in each set
classes = ['High squamous intra-epithelial lesion', 'Low squamous intra-epithelial lesion', 'Negative for Intraepithelial malignancy', 'Squamous cell carcinoma']

for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Split the data
def split_data(base_dir, train_dir, val_dir, test_dir, classes, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    for cls in classes:
        cls_dir = os.path.join(base_dir, cls)
        files = os.listdir(cls_dir)
        np.random.shuffle(files)

        train_count = int(len(files) * train_ratio)
        val_count = int(len(files) * val_ratio)

        train_files = files[:train_count]
        val_files = files[train_count:train_count + val_count]
        test_files = files[train_count + val_count:]

        for f in train_files:
            shutil.copy(os.path.join(cls_dir, f), os.path.join(train_dir, cls, f))

        for f in val_files:
            shutil.copy(os.path.join(cls_dir, f), os.path.join(val_dir, cls, f))

        for f in test_files:
            shutil.copy(os.path.join(cls_dir, f), os.path.join(test_dir, cls, f))

split_data(base_dir, train_dir, val_dir, test_dir, classes)


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image data generator without data augmentation
datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 937 images belonging to 5 classes.
Found 463 images belonging to 5 classes.
Found 267 images belonging to 5 classes.


In [16]:
os. rmdir('/content/data')

OSError: [Errno 39] Directory not empty: '/content/data'

In [ ]:
# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model_inception.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the Inception model
history_inception = model_inception.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20,
    callbacks=callbacks
)

# Change the ModelCheckpoint file name for ResNet model
callbacks[1] = ModelCheckpoint('best_model_resnet.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Train the ResNet model
history_resnet = model_resnet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: -211.0769 - accuracy: 0.1161
Epoch 1: val_loss improved from inf to -9330.54102, saving model to best_model_inception.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


21/21 [==============================] - 112s 3s/step - loss: -211.0769 - accuracy: 0.1161 - val_loss: -9330.5410 - val_accuracy: 0.1125
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: -1085.7385 - accuracy: 0.1176
Epoch 2: val_loss improved from -9330.54102 to -24932.77734, saving model to best_model_inception.h5
21/21 [==============================] - 58s 3s/step - loss: -1085.7385 - accuracy: 0.1176 - val_loss: -24932.7773 - val_accuracy: 0.1187
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: -2702.5156 - accuracy: 0.1176
Epoch 3: val_loss improved from -24932.77734 to -450673.00000, saving model to best_model_inception.h5
21/21 [==============================] - 56s 3s/step - loss: -2702.5156 - accuracy: 0.1176 - val_loss: -450673.0000 - val_accuracy: 0.1312
Epoch 4/20
21/21 [==============================] - ETA: 0s - loss: -5663.5693 - accuracy: 0.1176
Epoch 4: val_loss improved from -450673.00000 to -11435429.00000, saving model to best_m

KeyboardInterrupt: 

In [ ]:
# Load the best saved models
model_inception.load_weights('best_model_inception.h5')
model_resnet.load_weights('best_model_resnet.h5')

# Predict on the test data
preds_inception = model_inception.predict(test_generator)
preds_resnet = model_resnet.predict(test_generator)

# Average the predictions
final_preds = (preds_inception + preds_resnet) / 2.0
final_preds = np.round(final_preds)

# Evaluate the final ensemble model
print(classification_report(test_generator.classes, final_preds, target_names=['Non-cancerous', 'Cancerous']))
print(confusion_matrix(test_generator.classes, final_preds))
